In [33]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_vertexai import VertexAIEmbeddings, ChatVertexAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict
from typing import Iterator
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
import getpass
import os

In [34]:
#if not os.environ.get("GEMINI_API_KEY"):
  #os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter API key for GEMINI: ")

In [35]:
cards_loader = CSVLoader("./data/cards.csv")
doc1_loader = PyPDFLoader("./data/doc1.pdf")
doc2_loader = PyPDFLoader("./data/doc2.pdf")

In [36]:
cards = cards_loader.load()
doc1 = doc1_loader.load()
doc2 = doc2_loader.load()

In [37]:
csv_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0, add_start_index=True
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=20, add_start_index=True
)

In [38]:
card_splits = csv_splitter.split_documents(cards)
doc1_splits = text_splitter.split_documents(doc1)
doc2_splits = text_splitter.split_documents(doc2)

In [39]:
len(doc1_splits)

745

In [40]:
embeddings = HuggingFaceEmbeddings(model_name="ai-forever/sbert_large_nlu_ru")
vector_store = Chroma(embedding_function=embeddings, persist_directory="./vdb")

In [41]:
# _ = vector_store.add_documents(documents=card_splits)
print("Cards done")
# _ = vector_store.add_documents(documents=doc1_splits)
print("doc1 done")
# _ = vector_store.add_documents(documents=doc2_splits)
print("doc2 done")

Cards done
doc1 done
doc2 done


In [42]:
prompt = PromptTemplate.from_template(
    """
    Ты помощник-ассистент, который отвечает на вопросы клиента.
    Отвечай используя только следующий контекст.
    Если ты не знаешь или не нашел ответа, так и скажи.
    Если ты нашел несколько ответов, то расскажи о каждой из них.
    Вопрос: {question} 
    Контекст: {context} 
    Ответ:
    """
)

In [43]:
llm = ChatOllama(model="llama3.1:8b")

In [49]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    for doc in retrieved_docs:
        print(f"{doc.metadata}\n")
        print(f"{doc.page_content}\n")
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [50]:
result = graph.invoke({"question": "Сколько стоит оповещение об операциях"})
print(result["answer"])

{'row': 15, 'source': './data/cards.csv', 'start_index': 0}

Service: Плата за услугу «Оповещение об операциях»
Condition: в прочих случаях
Tariff: 99 руб. в месяц

{'row': 14, 'source': './data/cards.csv', 'start_index': 0}

Service: Плата за услугу «Оповещение об операциях»
Condition: с абонентским номером Тинькофф Мобайл
Tariff: 59 руб. в месяц

{'row': 13, 'source': './data/cards.csv', 'start_index': 0}

Service: Плата за обслуживание
Condition: в прочих случаях
Tariff: 99 руб. в месяц

{'row': 8, 'source': './data/cards.csv', 'start_index': 0}

Service: Пополнение
Condition: у партнеров Тинькофф до 150 000 руб. за расчетный период
Tariff: Бесплатно

По контексту я нашел несколько ответов:

1. В прочих случаях плата за услугу «Оповещение об операциях» составляет 99 рублей в месяц.
2. Если у клиента абонентский номер Тинькофф Мобайл, то плата за эту услугу составит 59 рублей в месяц.

Я не нашел информации о том, что плата за оповещение об операциях может быть бесплатной или имеет д